In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Abrimos el archivo con los nombres
dataset = open('../data/domain_names_full.txt', 'r').read().splitlines()[:10000] # Usamos solo el primer millon de dominios
dataset[:8]

In [ ]:
charset = ['*'] + sorted(list(set([y for x in dataset for y in x])))
ctoi = {c:i for i, c in enumerate(charset)}
itoc = {i:c for i, c in enumerate(charset)}
charset_len = len(charset)
print(ctoi)

In [ ]:
def build_dataset(dataset: list):
    X, Y  = [], []
    for d in dataset:
        example = list(d) + ['*']
        context = [0] * context_size
        for c in example:
            X.append(context)
            Y.append(ctoi[c])
            context = context[1:] + [ctoi[c]] 
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

In [ ]:
# build the dataset
context_size = 3
np.random.seed(42)
np.random.shuffle(dataset)
n1 = int(.8 * len(dataset))  # límite para el 80% del dataset
n2 = int(.9 * len(dataset))  # límite para el 90% del dataset
Xtr, Ytr = build_dataset(dataset[:n1])    # 80%
Xva, Yva = build_dataset(dataset[n1:n2])  # 10%
Xte, Yte = build_dataset(dataset[:n2])    # 10%

In [ ]:
# MLP de nuevo...
g = torch.Generator(device='cpu').manual_seed(42)
emb_d = 10  # El número de dimensiones del Enbedding
input_size = context_size * emb_d  # el tamaño del input, desapilado
n_hidden = 128  # El número de neuronas en al capa hiddend que queremos

# Definición del modelo
C = torch.randn((charset_len, emb_d),             generator=g)
W1 = torch.randn((emb_d * context_size, n_hidden),generator=g)
b1 = torch.randn(n_hidden,                        generator=g)
W2 = torch.randn((n_hidden, charset_len),         generator=g)
b2 = torch.randn(charset_len,                     generator=g)
parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True
sum(p.nelement() for p in parameters)

In [ ]:
train_iterations = 200000
minibatch_size = 64
loss_log = []

for i in range(train_iterations):
    # training loop en mini batches
    ix = torch.randint(0, len(Xtr), (minibatch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]
    # forward pass usando F.cross_entropy
    emb = C[Xb]  # embedding de los caracteres
    embcat = emb.view(-1, input_size)  # embedding como una vector de input_size
    hpreact = embcat @ W1 + b1  # pre activación de la capa oculta (h)
    h = torch.tanh(hpreact)  # activación de la capa oculta (h)
    logits = h @ W2 + b2  # output layer
    loss = F.cross_entropy(logits, Yb)
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    lr = .1 if i < 100000 else .01
    for p in parameters:
        p.data += -lr * p.grad
    if i % 10000 == 0:
        print(f'Step: {i:7d}/{train_iterations:7d} -- loss: {loss.item():.6f}')
    loss_log.append(loss.log10().item())
print(loss.item())

In [ ]:
plt.plot(loss_log)

In [ ]:
plt.hist(h.view(-1).tolist(),bins=50)

In [ ]:
plt.imshow(h.abs() > 0.99, cmap='gray')

In [ ]:
class Embedding():
    def __init__(self, n_embeddings, embedding_dimension):
        self.weight = torch.randn((n_embeddings, embedding_dimension))
    
    def __call__(self, x):
        self.out = self.weight[x]
        self.out = self.out.view(self.out.shape[0], -1)
        return self.out

    def parameters(self):
        return [self.weight]


class Linear():
    def __init__(self, fan_in, fan_out, bias=True):
        self.weight = torch.randn((fan_in, fan_out))
        self.bias = torch.zeros(fan_out) if bias else None
    
    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])


class Tanh():
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    
    def parameters(self):
        return []

## A practicar!
El siguiente código tiene algunos errores. En pricipio, el loop de training no aprende nada (el loss no baja) más allá del segundo loop. Encuentren el error.

In [ ]:
context_size = 3
emb_d = 10
n_hidden = 128

model = [
    Embedding(charset_len, emb_d),
    Linear(emb_d*context_size, n_hidden), Tanh(),
    Linear(n_hidden, charset_len)
]

parameters = [p for layer in model for p in layer.parameters()]
for p in parameters:
    p.requires_grad = True
print(f'Number of parameters: {sum((p.nelement() for p in parameters))}')

In [ ]:
train_iterations = 1000000
minibatch_size = 64
loss_log = []

for i in range(train_iterations):
    # training loop en mini batches
    ix = torch.randint(0, len(Xtr), (minibatch_size,))
    Xb, Yb = Xtr[ix], Ytr[ix]
    x = Xb
    # forward pass
    for layer in model:
        x = layer(x)
    loss = F.cross_entropy(x, Yb)
    
    # backward pass
    for layer in model:
        layer.out.retain_grad()
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # update
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data *= -lr * p.grad
    
    if i % 10000 == 0:
        print(f'Step: {i:7d}/{train_iterations:7d} -- loss: {loss.item():.6f}')
    loss_log.append(loss.log10().item())
print(loss.item())
    

### Haciendo inferencia
Una vez entrenado el modelo, hay que escribir una función para samplear del modelo.

In [ ]:
# Función para samplear del modelo

### Agregando capas
Nuestro modelo tiene una capa sola pero ahora agregar capas es re facil!! 

El siguiente modelo tiene varias capas. Analicen como cambiarían el training loop y la función de inferencia para que el modelo siga funcionando.

In [ ]:
model = [
    Embedding(charset_len, emb_d),
    Linear(emb_d*context_size, n_hidden), Tanh(),
    Linear(emb_d*context_size, n_hidden), Tanh(),
    Linear(emb_d*context_size, n_hidden), Tanh(),
    Linear(emb_d*context_size, n_hidden), Tanh(),
    Linear(n_hidden, charset_len)
]

### Diagnóstico y análisis
Estaría bueno poder mostrar la saturación de la función de activación y algunos otros plots de diagnostico.

Modifiquen el modelo anterior para que tenga 7 capas ocultas. Analicen que pasa con las funciones de activación

In [17]:
# Train/Validation loss plot

In [18]:
# Saturación de las neuronas de las diferentes capas

In [19]:
# Histograma de saturacion de las capas (a la Karpathy https://youtu.be/P6sfmUTpUmc?feature=shared&t=5242)